# Plugin Manager

> Plugin discovery, loading, and lifecycle management system

In [ ]:
#| default_exp core.manager

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
import time
from typing import Any, Dict, List, Optional, Type

from cjm_plugin_system.core.config import get_config
from cjm_plugin_system.core.interface import PluginInterface
from cjm_plugin_system.core.metadata import PluginMeta
from cjm_plugin_system.core.proxy import RemotePluginProxy
from cjm_plugin_system.core.scheduling import ResourceScheduler, PermissiveScheduler

import logging
logging.basicConfig(
    level=logging.INFO,
    format='[PluginManager] %(message)s',
    force=True
)

## PluginManager

The `PluginManager` orchestrates the complete lifecycle of plugins in the process-isolated architecture:

- **Discovery**: Finds plugin manifests in local (`.cjm/plugins/`) or global (`~/.cjm/plugins/`) directories
- **Loading**: Creates `RemotePluginProxy` instances that spawn isolated Worker subprocesses
- **Execution**: Forwards calls to Workers via HTTP, supports both sync and async
- **Lifecycle**: Handles initialization, configuration updates, and cleanup

```
PluginManager                           Worker Subprocesses
┌─────────────────┐                    ┌─────────────────────┐
│ discover_       │                    │ Conda Env: Whisper  │
│   manifests()   │     HTTP/JSON      │   └─ WhisperPlugin  │
│                 │◄──────────────────▶│                     │
│ plugins:        │                    └─────────────────────┘
│   whisper ──────┼──► RemoteProxy     ┌─────────────────────┐
│   gemini ───────┼──► RemoteProxy ◄──▶│ Conda Env: Gemini   │
│                 │                    │   └─ GeminiPlugin   │
└─────────────────┘                    └─────────────────────┘
```

In [ ]:
#| export
class PluginManager:
    """Manages plugin discovery, loading, and lifecycle via process isolation."""

    def __init__(
        self,
        plugin_interface:Type[PluginInterface]=PluginInterface, # Base interface for type checking
        search_paths:Optional[List[Path]]=None, # Custom manifest search paths
        scheduler:Optional[ResourceScheduler]=None # Resource allocation policy
    ):
        """Initialize the plugin manager."""
        self.plugin_interface = plugin_interface
        
        # Use config-based search paths if not explicitly provided
        if search_paths is None:
            cfg = get_config()
            self.search_paths = [
                Path.cwd() / ".cjm" / "plugins",  # Local (high priority)
                cfg.plugins_dir                    # Config-based (replaces ~/.cjm/plugins)
            ]
        else:
            self.search_paths = search_paths
        
        self.scheduler = scheduler or PermissiveScheduler()
        self.system_monitor: Optional[PluginInterface] = None
        self.discovered: List[PluginMeta] = []
        self.plugins: Dict[str, PluginMeta] = {}
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")

    def register_system_monitor(
        self,
        plugin_name:str # Name of the system monitor plugin
    ) -> None:
        """Bind a loaded plugin to act as the hardware system monitor."""
        self.system_monitor = self.get_plugin(plugin_name)
        if self.system_monitor:
            self.logger.info(f"Registered system monitor: {plugin_name}")
        else:
            self.logger.warning(f"System monitor plugin not found: {plugin_name}")

    def _get_global_stats(self) -> Dict[str, Any]: # Current system telemetry
        """Fetch real-time stats from the system monitor plugin (sync)."""
        if self.system_monitor:
            try:
                return self.system_monitor.execute("get_system_status")
            except Exception as e:
                self.logger.warning(f"Failed to fetch system stats: {e}")
        return {}

    async def _get_global_stats_async(self) -> Dict[str, Any]: # Current system telemetry
        """Fetch real-time stats from the system monitor plugin (async)."""
        if self.system_monitor:
            try:
                return await self.system_monitor.execute_async("get_system_status")
            except Exception as e:
                self.logger.warning(f"Failed to fetch system stats: {e}")
        return {}

    def discover_manifests(self) -> List[PluginMeta]: # List of discovered plugin metadata
        """Discover plugins via JSON manifests in search paths."""
        self.discovered = []
        seen_plugins = set()

        for base_path in self.search_paths:
            if not base_path.exists():
                continue
            
            for manifest_file in base_path.glob("*.json"):
                try:
                    with open(manifest_file) as f:
                        manifest = json.load(f)
                    
                    name = manifest.get('name')
                    if not name or name in seen_plugins:
                        continue  # Skip duplicates (local shadows global)
                    
                    # Create metadata with manifest attached
                    meta = PluginMeta(
                        name=name,
                        version=manifest.get('version', '0.0.0'),
                        description=manifest.get('description', ''),
                        author=manifest.get('author', ''),
                        package_name=manifest.get('module', ''),
                        category=manifest.get('category', ''),
                        interface=manifest.get('interface', ''),
                        config_schema=manifest.get('config_schema')
                    )
                    meta.manifest = manifest
                    
                    self.discovered.append(meta)
                    seen_plugins.add(name)
                    self.logger.info(f"Discovered manifest: {name} from {manifest_file}")

                except Exception as e:
                    self.logger.error(f"Error loading manifest {manifest_file}: {e}")

        return self.discovered

    def get_discovered_by_category(
        self,
        category:str # Category to filter by (e.g., "transcription")
    ) -> List[PluginMeta]: # List of matching discovered plugins
        """Get discovered plugins filtered by category."""
        return [meta for meta in self.discovered if meta.category == category]

    def get_plugins_by_category(
        self,
        category:str # Category to filter by (e.g., "transcription")
    ) -> List[PluginMeta]: # List of matching loaded plugins
        """Get loaded plugins filtered by category."""
        return [meta for meta in self.plugins.values() if meta.category == category]

    def get_discovered_categories(self) -> List[str]: # List of unique categories
        """Get all unique categories among discovered plugins."""
        return list(set(meta.category for meta in self.discovered if meta.category))

    def get_loaded_categories(self) -> List[str]: # List of unique categories
        """Get all unique categories among loaded plugins."""
        return list(set(meta.category for meta in self.plugins.values() if meta.category))

    def get_plugin_meta(
        self,
        plugin_name:str # Name of the plugin
    ) -> Optional[PluginMeta]: # Plugin metadata or None
        """Get metadata for a loaded plugin by name."""
        return self.plugins.get(plugin_name)

    def get_discovered_meta(
        self,
        plugin_name:str # Name of the plugin
    ) -> Optional[PluginMeta]: # Plugin metadata or None
        """Get metadata for a discovered (not necessarily loaded) plugin by name."""
        for meta in self.discovered:
            if meta.name == plugin_name:
                return meta
        return None

    def _extract_defaults_from_schema(
        self,
        config_schema:Optional[Dict[str, Any]] # JSON Schema with properties
    ) -> Dict[str, Any]: # Default values extracted from schema
        """Extract default values from a JSON Schema's properties."""
        if not config_schema:
            return {}

        properties = config_schema.get("properties", {})
        defaults = {}

        for field_name, field_schema in properties.items():
            if "default" in field_schema:
                defaults[field_name] = field_schema["default"]

        return defaults

    def load_plugin(
        self,
        plugin_meta:PluginMeta, # Plugin metadata (with manifest attached)
        config:Optional[Dict[str, Any]]=None # Initial configuration
    ) -> bool: # True if successfully loaded
        """Load a plugin by spawning a Worker subprocess."""
        if not hasattr(plugin_meta, 'manifest'):
            self.logger.error(f"Plugin {plugin_meta.name} has no manifest data")
            return False

        try:
            self.logger.info(f"Launching worker for {plugin_meta.name}...")
            proxy = RemotePluginProxy(plugin_meta.manifest)

            # If config is None or empty, extract defaults from the plugin's config schema
            if not config:
                config_schema = plugin_meta.manifest.get("config_schema")
                config = self._extract_defaults_from_schema(config_schema)
                if config:
                    self.logger.info(f"Using default config for {plugin_meta.name}: {list(config.keys())}")

            # Initialize with config (defaults or provided)
            if config:
                proxy.initialize(config)

            plugin_meta.instance = proxy
            self.plugins[plugin_meta.name] = plugin_meta
            self.logger.info(f"Loaded plugin: {plugin_meta.name}")
            return True

        except Exception as e:
            self.logger.error(f"Failed to load plugin {plugin_meta.name}: {e}")
            return False

    def load_all(
        self,
        configs:Optional[Dict[str, Dict[str, Any]]]=None # Plugin name -> config mapping
    ) -> Dict[str, bool]: # Plugin name -> success mapping
        """Discover and load all available plugins."""
        configs = configs or {}
        results = {}
        
        self.discover_manifests()
        for meta in self.discovered:
            config = configs.get(meta.name)
            results[meta.name] = self.load_plugin(meta, config)
        
        return results

    def unload_plugin(
        self,
        plugin_name:str # Name of the plugin to unload
    ) -> bool: # True if successfully unloaded
        """Unload a plugin and terminate its Worker process."""
        if plugin_name not in self.plugins:
            self.logger.error(f"Plugin {plugin_name} not found")
            return False

        try:
            plugin_meta = self.plugins[plugin_name]
            if plugin_meta.instance:
                plugin_meta.instance.cleanup()

            del self.plugins[plugin_name]
            self.logger.info(f"Unloaded plugin: {plugin_name}")
            return True

        except Exception as e:
            self.logger.error(f"Error unloading plugin {plugin_name}: {e}")
            return False

    def unload_all(self) -> None:
        """Unload all plugins and terminate all Worker processes."""
        for name in list(self.plugins.keys()):
            self.unload_plugin(name)

    def get_plugin(
        self,
        plugin_name:str # Name of the plugin
    ) -> Optional[PluginInterface]: # Plugin proxy instance or None
        """Get a loaded plugin instance by name."""
        if plugin_name in self.plugins:
            return self.plugins[plugin_name].instance
        return None

    def list_plugins(self) -> List[PluginMeta]: # List of loaded plugin metadata
        """List all loaded plugins."""
        return list(self.plugins.values())

    def _evict_for_resources(self, needed_meta:PluginMeta) -> bool:
        """Attempt to free resources by unloading/releasing idle plugins (LRU)."""
        self.logger.info(f"Attempting eviction to make room for {needed_meta.name}...")
        
        # 1. Identify Candidates: Running plugins that ARE NOT the one we need
        candidates = [
            meta for name, meta in self.plugins.items()
            if meta.instance 
            and name != needed_meta.name
            and meta.manifest.get('resources', {}).get('requires_gpu', False) # Only evict GPU users
        ]
        
        # 2. Sort by LRU (Oldest timestamp first)
        candidates.sort(key=lambda x: x.last_executed)
        
        # 3. Evict one by one until we have space
        for candidate in candidates:
            self.logger.info(f"Evicting idle plugin: {candidate.name} (Last used: {candidate.last_executed})")
            
            # Prefer Soft Release if available, else Hard Unload
            if hasattr(candidate.instance, 'release'):
                candidate.instance.release()
            else:
                self.reload_plugin(candidate.name)
            
            # Wait a moment for VRAM to clear
            time.sleep(0.5) 
            
            # Re-check scheduler immediately
            # If allocate returns True now, we stop evicting
            if self.scheduler.allocate(needed_meta, self._get_global_stats):
                return True
                
        return False

    def execute_plugin(
        self,
        plugin_name:str, # Name of the plugin
        *args,
        **kwargs
    ) -> Any: # Plugin result
        """Execute a plugin's main functionality (sync)."""
        plugin = self.get_plugin(plugin_name)
        if not plugin:
            raise ValueError(f"Plugin {plugin_name} not found or not loaded")

        if not self.plugins[plugin_name].enabled:
            raise ValueError(f"Plugin {plugin_name} is disabled")

        # Update Timestamp (Mark as active)
        plugin_meta = self.plugins[plugin_name]
        plugin_meta.last_executed = time.time()

        # Allocation Loop
        stats_provider = self._get_global_stats

        # Scheduling check (pass method reference for polling support)
        if not self.scheduler.allocate(plugin_meta, stats_provider):
            # Trigger Eviction if blocked
            self.logger.warning(f"Resources busy for {plugin_name}. Triggering eviction protocol.")
            
            if self._evict_for_resources(plugin_meta):
                self.logger.info("Eviction successful. Resources acquired.")
            else:
                # If eviction failed (or QueueScheduler timed out), raise error
                raise RuntimeError(f"ResourceScheduler blocked execution of {plugin_name} (Eviction failed)")

        self.scheduler.on_execution_start(plugin_name)
        try:
            return plugin.execute(*args, **kwargs)
        finally:
            self.scheduler.on_execution_finish(plugin_name)

    async def execute_plugin_async(
        self,
        plugin_name:str, # Name of the plugin
        *args,
        **kwargs
    ) -> Any: # Plugin result
        """Execute a plugin's main functionality (async)."""
        plugin = self.get_plugin(plugin_name)
        if not plugin:
            raise ValueError(f"Plugin {plugin_name} not found or not loaded")

        if not self.plugins[plugin_name].enabled:
            raise ValueError(f"Plugin {plugin_name} is disabled")

        # Update Timestamp (Mark as active)
        plugin_meta = self.plugins[plugin_name]
        plugin_meta.last_executed = time.time()

        # Allocation Loop
        stats_provider = self._get_global_stats

        # Async scheduling check (pass async method for non-blocking polling)
        if not await self.scheduler.allocate_async(self.plugins[plugin_name], self._get_global_stats_async):
            # Trigger Eviction if blocked
            self.logger.warning(f"Resources busy for {plugin_name}. Triggering eviction protocol.")
            
            if self._evict_for_resources(plugin_meta):
                self.logger.info("Eviction successful. Resources acquired.")
            else:
                # If eviction failed (or QueueScheduler timed out), raise error
                raise RuntimeError(f"ResourceScheduler blocked execution of {plugin_name} (Eviction failed)")

        self.scheduler.on_execution_start(plugin_name)
        try:
            return await plugin.execute_async(*args, **kwargs)
        finally:
            self.scheduler.on_execution_finish(plugin_name)

    def enable_plugin(
        self,
        plugin_name:str # Name of the plugin
    ) -> bool: # True if plugin was enabled
        """Enable a plugin."""
        if plugin_name in self.plugins:
            self.plugins[plugin_name].enabled = True
            return True
        return False

    def disable_plugin(
        self,
        plugin_name:str # Name of the plugin
    ) -> bool: # True if plugin was disabled
        """Disable a plugin without unloading it."""
        if plugin_name in self.plugins:
            self.plugins[plugin_name].enabled = False
            return True
        return False

    def get_plugin_logs(
        self,
        plugin_name:str, # Name of the plugin
        lines:int=50 # Number of lines to return
    ) -> str: # Log content
        """Read the last N lines of the plugin's log file."""
        cfg = get_config()
        log_path = cfg.logs_dir / f"{plugin_name}.log"
        
        if not log_path.exists():
            return "No logs found."
            
        try:
            # Efficient tail implementation
            from collections import deque
            with open(log_path, 'r') as f:
                tail = deque(f, maxlen=lines)
                return "".join(tail)
        except Exception as e:
            return f"Error reading logs: {e}"

Key features:

- **Local-first discovery**: Manifests in `.cjm/plugins/` shadow global ones in `~/.cjm/plugins/`
- **Process isolation**: Each plugin runs in its own subprocess with a dedicated Python interpreter
- **Dual execution modes**: `execute_plugin()` for sync, `execute_plugin_async()` for async
- **Automatic cleanup**: `unload_all()` terminates all Worker processes

## Configuration Management

Methods for managing plugin configuration. These forward to the `RemotePluginProxy` which communicates with the Worker over HTTP.

In [ ]:
#| export
def get_plugin_config(
    self,
    plugin_name: str # Name of the plugin
) -> Optional[Dict[str, Any]]: # Current configuration or None
    """Get the current configuration of a plugin."""
    plugin = self.get_plugin(plugin_name)
    if plugin:
        return plugin.get_current_config()
    return None

def get_plugin_config_schema(
    self,
    plugin_name: str # Name of the plugin
) -> Optional[Dict[str, Any]]: # JSON Schema or None
    """Get the configuration JSON Schema for a plugin."""
    plugin = self.get_plugin(plugin_name)
    if plugin:
        return plugin.get_config_schema()
    return None

def get_all_plugin_configs(self) -> Dict[str, Dict[str, Any]]: # Plugin name -> config mapping
    """Get current configuration for all loaded plugins."""
    return {
        name: plugin.get_current_config()
        for name, meta in self.plugins.items()
        if meta.instance
        for plugin in [meta.instance]
    }

def update_plugin_config(
    self,
    plugin_name: str, # Name of the plugin
    config: Dict[str, Any] # New configuration values
) -> bool: # True if successful
    """Update a plugin's configuration (hot-reload without restart)."""
    plugin = self.get_plugin(plugin_name)
    if not plugin:
        self.logger.error(f"Plugin {plugin_name} not found")
        return False

    try:
        plugin.initialize(config)
        self.logger.info(f"Updated configuration for plugin: {plugin_name}")
        return True
    except Exception as e:
        self.logger.error(f"Error updating plugin {plugin_name} config: {e}")
        return False

def reload_plugin(
    self,
    plugin_name: str, # Name of the plugin
    config: Optional[Dict[str, Any]] = None # Optional new configuration
) -> bool: # True if successful
    """Reload a plugin by terminating and restarting its Worker."""
    if plugin_name not in self.plugins:
        self.logger.error(f"Plugin {plugin_name} not found")
        return False

    try:
        plugin_meta = self.plugins[plugin_name]
        
        # Get current config if not provided
        if config is None and plugin_meta.instance:
            config = plugin_meta.instance.get_current_config()
        
        # Unload and reload
        self.unload_plugin(plugin_name)
        return self.load_plugin(plugin_meta, config)

    except Exception as e:
        self.logger.error(f"Error reloading plugin {plugin_name}: {e}")
        return False

def get_plugin_stats(
    self,
    plugin_name: str # Name of the plugin
) -> Optional[Dict[str, Any]]: # Resource telemetry or None
    """Get resource usage stats for a plugin's Worker process."""
    plugin = self.get_plugin(plugin_name)
    if plugin and hasattr(plugin, 'get_stats'):
        return plugin.get_stats()
    return None

# Add methods to PluginManager
PluginManager.get_plugin_config = get_plugin_config
PluginManager.get_plugin_config_schema = get_plugin_config_schema
PluginManager.get_all_plugin_configs = get_all_plugin_configs
PluginManager.update_plugin_config = update_plugin_config
PluginManager.reload_plugin = reload_plugin
PluginManager.get_plugin_stats = get_plugin_stats

## Streaming Execution

Async streaming support for real-time results (e.g., transcription word-by-word).

In [ ]:
#| export
from typing import AsyncGenerator

async def execute_plugin_stream(
    self,
    plugin_name: str,  # Name of the plugin
    *args,
    **kwargs
) -> AsyncGenerator[Any, None]:  # Async generator yielding results
    """Execute a plugin with streaming response."""
    plugin = self.get_plugin(plugin_name)
    if not plugin:
        raise ValueError(f"Plugin {plugin_name} not found or not loaded")

    if not self.plugins[plugin_name].enabled:
        raise ValueError(f"Plugin {plugin_name} is disabled")

    # Async scheduling check (pass async method for non-blocking polling)
    if not await self.scheduler.allocate_async(self.plugins[plugin_name], self._get_global_stats_async):
        raise RuntimeError(f"ResourceScheduler blocked execution of {plugin_name}")

    self.scheduler.on_execution_start(plugin_name)
    try:
        async for chunk in plugin.execute_stream(*args, **kwargs):
            yield chunk
    finally:
        self.scheduler.on_execution_finish(plugin_name)

# Add to PluginManager
PluginManager.execute_plugin_stream = execute_plugin_stream

## Usage Examples

### Basic Usage

```python
import logging
from cjm_plugin_system.core.manager import PluginManager

logging.basicConfig(level=logging.INFO)

# Create manager
manager = PluginManager()

# Discover and load all plugins from manifest directories
results = manager.load_all()
print(f"Loaded: {results}")

# Execute a plugin
result = manager.execute_plugin("whisper-local", audio="/path/to/audio.wav")

# Update configuration (hot-reload)
manager.update_plugin_config("whisper-local", {"model": "large-v3"})

# Clean up all workers
manager.unload_all()
```

### Async Usage (FastHTML)

```python
async def transcribe(audio_path: str):
    manager = PluginManager()
    manager.load_all()
    
    try:
        result = await manager.execute_plugin_async("whisper-local", audio=audio_path)
        return result
    finally:
        manager.unload_all()

# Streaming
async def transcribe_stream(audio_path: str):
    manager = PluginManager()
    manager.load_all()
    
    try:
        async for chunk in manager.execute_plugin_stream("whisper-local", audio=audio_path):
            yield chunk
    finally:
        manager.unload_all()
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()